In [1]:
import pandas as pd

In [2]:
file=r'/Users/Monica/Desktop/salary_prediction_app/eda_data.csv'

In [3]:
train=pd.read_csv(file)

In [4]:
train=train.iloc[:,1:]

In [5]:
import re

In [6]:
k=[re.sub('[$A-Za-z\(\)\.:]','',x) for x in train['Salary Estimate']]

In [7]:
train['Salary Estimate']=pd.Series(k).str.split('-',expand=True).astype(float).mean(axis=1)

In [8]:
train['Company Name']=train['Company Name'].str.split('\n',expand=True).iloc[:,0]

In [9]:
train.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn',
       'spark', 'aws', 'excel', 'job_simp', 'seniority', 'desc_len',
       'num_comp'],
      dtype='object')

In [10]:
# for simplyfying things , you can later on make use of these if you like
# except the salary vars
drop_cols=['Job Description','seniority','job_simp','min_salary','max_salary','avg_salary','company_txt','desc_len']

In [11]:
train.drop(drop_cols,1,inplace=True)

In [12]:
train.shape

(742, 24)

In [13]:
train['Size'].value_counts()

1001 to 5000 employees     150
501 to 1000 employees      134
10000+ employees           130
201 to 500 employees       117
51 to 200 employees         94
5001 to 10000 employees     76
1 to 50 employees           31
Unknown                      9
-1                           1
Name: Size, dtype: int64

In [14]:
train.to_csv('simple_data.csv',index=False)

In [15]:
from mypipes import *

In [16]:
cat_vars=list(train.select_dtypes(['object']).columns)

In [17]:
num_vars=list(train.select_dtypes(exclude=['object']).columns)
num_vars.remove('Salary Estimate')

In [18]:
cat_vars

['Job Title',
 'Company Name',
 'Location',
 'Headquarters',
 'Size',
 'Type of ownership',
 'Industry',
 'Sector',
 'Revenue',
 'Competitors',
 'job_state']

In [19]:
cat_pipe=Pipeline([
    ('cat_var_select',VarSelector(cat_vars)),
    ('create_dummies',get_dummies_Pipe(74))
])

In [20]:
data_pipe=FeatureUnion([
    ('num_vars',VarSelector(num_vars)),
    ('cat_data',cat_pipe)
])

In [21]:
from sklearn.linear_model import LinearRegression
# fitting a simple model , you can later make something complex and tuned

In [22]:
complete_pipe=Pipeline([
    ('data_pipe',data_pipe),
    ('model',LinearRegression(fit_intercept=True))
])

In [23]:
target='Salary Estimate'
y=train[target]
x=train.drop(target,1)

In [24]:
complete_pipe.fit(x,y)

Pipeline(memory=None,
     steps=[('data_pipe', FeatureUnion(n_jobs=None,
       transformer_list=[('num_vars', VarSelector(drop_var=False, var_names=None)), ('cat_data', Pipeline(memory=None,
     steps=[('cat_var_select', VarSelector(drop_var=False, var_names=None)), ('create_dummies', get_dummies_Pipe(freq_cutoff=74))]))],
       transformer_weights=None)), ('model', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))])

In [25]:
from sklearn.externals import joblib
joblib.dump(complete_pipe,'model_pipeline.pkl')

['model_pipeline.pkl']

In [26]:
x[num_vars].describe()

,Rating,Founded,hourly,employer_provided,same_state,age,python_yn,R_yn,spark,aws,excel,num_comp
count,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000
mean,3.618868,1837.154987,0.032345,0.022911,0.557951,46.591644,0.528302,0.002695,0.225067,0.237197,0.522911,1.053908
std,0.801210,497.183763,0.177034,0.149721,0.496965,53.778815,0.499535,0.051882,0.417908,0.425651,0.499812,1.384239
min,-1.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.300000,1939.000000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.700000,1988.000000,0.000000,0.000000,1.000000,24.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,4.000000,2007.000000,0.000000,0.000000,1.000000,59.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000
max,5.000000,2019.000000,1.000000,1.000000,1.000000,276.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000


In [27]:
x.columns

Index(['Job Title', 'Rating', 'Company Name', 'Location', 'Headquarters',
       'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue',
       'Competitors', 'hourly', 'employer_provided', 'job_state', 'same_state',
       'age', 'python_yn', 'R_yn', 'spark', 'aws', 'excel', 'num_comp'],
      dtype='object')